In [30]:
import numpy as np

import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.autograd import Variable

## Turn each input into a vector of "activations

In [16]:
def binary_encode(i, num_digits):
    """将每个input转换为binary digits(转换为二进制的表示, 最多可是表示2^num_digits)
    """
    return np.array([i >> d & 1 for d in range(num_digits)])

In [17]:
binary_encode(10,10)

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0])

## Turn each output into a vector

In [69]:
def fizz_buzz_encode(i):
    """将output转换为lebel
    """
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else:             return 0

In [70]:
fizz_buzz_encode(30)

3

## Get the training data(101-1024)

In [116]:
NUM_DIGITS = 10
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2**NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i) for i in range(101, 2**NUM_DIGITS)])

In [72]:
trX[:5]

array([[1, 0, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 1, 0, 0, 0],
       [1, 1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 1, 1, 0, 0, 0]])

In [109]:
trY[:5]

array([0, 1, 0, 0, 3])

## Generate the Model

In [201]:
class FizzBuzzModel(nn.Module):
    def __init__(self, in_features, out_classes, hidden_size, n_hidden_layers):
        super(FizzBuzzModel,self).__init__()
        layers = []
        for i in range(n_hidden_layers):
            layers.append(nn.Linear(hidden_size,hidden_size))
            # layers.append(nn.Dropout(0.5))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.ReLU())
        self.inputLayer = nn.Linear(in_features, hidden_size)
        self.relu = nn.ReLU()
        self.layers = nn.Sequential(*layers)
        self.outputLayer = nn.Linear(hidden_size, out_classes)
        
    def forward(self,x):
        x = self.inputLayer(x)
        x = self.relu(x)
        x = self.layers(x)
        out = self.outputLayer(x)
        return out

## Train the Model

In [202]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# define the model
simpleModel = FizzBuzzModel(10,4,150,3).to(device)
print(simpleModel)
# Loss and optimizer
learning_rate = 0.05
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(simpleModel.parameters(), lr=learning_rate)

FizzBuzzModel(
  (inputLayer): Linear(in_features=10, out_features=150, bias=True)
  (relu): ReLU()
  (layers): Sequential(
    (0): Linear(in_features=150, out_features=150, bias=True)
    (1): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=150, out_features=150, bias=True)
    (4): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=150, out_features=150, bias=True)
    (7): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (outputLayer): Linear(in_features=150, out_features=4, bias=True)
)


In [203]:
# 使用batch进行训练
FizzBuzzDataset = Data.TensorDataset(torch.from_numpy(trX).float().to(device),
                                    torch.from_numpy(trY).long().to(device))
loader = Data.DataLoader(dataset=FizzBuzzDataset,
                        batch_size=128*5,
                        shuffle=True)

In [165]:
for batch_x,batch_y in loader:
    print(batch_x[:5])
    print(batch_x.shape)
    print(batch_y[:5])
    break

tensor([[0., 1., 1., 1., 1., 0., 0., 1., 1., 0.],
        [1., 0., 1., 1., 0., 0., 1., 0., 0., 1.],
        [0., 1., 0., 0., 0., 1., 0., 1., 1., 1.],
        [0., 0., 1., 1., 0., 1., 0., 0., 0., 1.],
        [0., 0., 1., 1., 0., 1., 1., 0., 0., 0.]], device='cuda:0')
torch.Size([640, 10])
tensor([1, 0, 3, 0, 1], device='cuda:0')


In [204]:
# 进行训练
simpleModel.train()
epochs = 3000
for epoch in range(1,epochs):
    for step,(batch_x, batch_y) in enumerate(loader):
        out = simpleModel(batch_x) # 前向传播
        loss = criterion(out, batch_y) # 计算损失
        
        optimizer.zero_grad() # 梯度清零
        loss.backward() # 反向传播
        optimizer.step() # 随机梯度下降
        
    correct = 0
    total = 0
    _, predicted = torch.max(out.data, 1)
    total += batch_y.size(0)
    correct += (predicted == batch_y).sum().item()
    acc = 100*correct/total
    print('Epoch : {:0>4d} | Loss : {:<6.4f} | Train Accuracy : {:<6.2f}%'.format(epoch,loss,acc))

Epoch : 0001 | Loss : 1.3540 | Train Accuracy : 48.41 %
Epoch : 0002 | Loss : 1.5489 | Train Accuracy : 44.17 %
Epoch : 0003 | Loss : 1.1712 | Train Accuracy : 53.71 %
Epoch : 0004 | Loss : 1.1831 | Train Accuracy : 55.12 %
Epoch : 0005 | Loss : 1.1497 | Train Accuracy : 53.71 %
Epoch : 0006 | Loss : 1.1409 | Train Accuracy : 53.00 %
Epoch : 0007 | Loss : 1.0948 | Train Accuracy : 52.30 %
Epoch : 0008 | Loss : 1.1374 | Train Accuracy : 49.47 %
Epoch : 0009 | Loss : 1.0247 | Train Accuracy : 54.77 %
Epoch : 0010 | Loss : 0.9732 | Train Accuracy : 55.48 %
Epoch : 0011 | Loss : 1.0046 | Train Accuracy : 53.00 %
Epoch : 0012 | Loss : 0.9133 | Train Accuracy : 55.48 %
Epoch : 0013 | Loss : 0.8898 | Train Accuracy : 55.12 %
Epoch : 0014 | Loss : 0.8197 | Train Accuracy : 58.66 %
Epoch : 0015 | Loss : 0.7565 | Train Accuracy : 60.42 %
Epoch : 0016 | Loss : 0.7367 | Train Accuracy : 65.02 %
Epoch : 0017 | Loss : 0.6759 | Train Accuracy : 62.90 %
Epoch : 0018 | Loss : 0.5952 | Train Accuracy : 

Epoch : 0149 | Loss : 0.0008 | Train Accuracy : 100.00%
Epoch : 0150 | Loss : 0.0022 | Train Accuracy : 100.00%
Epoch : 0151 | Loss : 0.0098 | Train Accuracy : 99.65 %
Epoch : 0152 | Loss : 0.0162 | Train Accuracy : 99.65 %
Epoch : 0153 | Loss : 0.0010 | Train Accuracy : 100.00%
Epoch : 0154 | Loss : 0.0022 | Train Accuracy : 100.00%
Epoch : 0155 | Loss : 0.0046 | Train Accuracy : 100.00%
Epoch : 0156 | Loss : 0.0007 | Train Accuracy : 100.00%
Epoch : 0157 | Loss : 0.0066 | Train Accuracy : 99.65 %
Epoch : 0158 | Loss : 0.0019 | Train Accuracy : 100.00%
Epoch : 0159 | Loss : 0.0015 | Train Accuracy : 100.00%
Epoch : 0160 | Loss : 0.0014 | Train Accuracy : 100.00%
Epoch : 0161 | Loss : 0.0004 | Train Accuracy : 100.00%
Epoch : 0162 | Loss : 0.0054 | Train Accuracy : 100.00%
Epoch : 0163 | Loss : 0.0007 | Train Accuracy : 100.00%
Epoch : 0164 | Loss : 0.0116 | Train Accuracy : 99.65 %
Epoch : 0165 | Loss : 0.0002 | Train Accuracy : 100.00%
Epoch : 0166 | Loss : 0.0003 | Train Accuracy : 

Epoch : 0299 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0300 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0301 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0302 | Loss : 0.0002 | Train Accuracy : 100.00%
Epoch : 0303 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0304 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0305 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0306 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0307 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0308 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0309 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0310 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0311 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0312 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0313 | Loss : 0.0002 | Train Accuracy : 100.00%
Epoch : 0314 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0315 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0316 | Loss : 0.0000 | Train Accuracy : 

Epoch : 0447 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0448 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0449 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0450 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0451 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0452 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0453 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0454 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0455 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0456 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0457 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0458 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0459 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0460 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0461 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0462 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0463 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0464 | Loss : 0.0000 | Train Accuracy : 

Epoch : 0594 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0595 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0596 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 0597 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0598 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0599 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0600 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0601 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0602 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0603 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0604 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0605 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0606 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0607 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0608 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0609 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0610 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 0611 | Loss : 0.0000 | Train Accuracy : 

...

Epoch : 2375 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2376 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2377 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2378 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2379 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2380 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2381 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2382 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2383 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2384 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2385 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2386 | Loss : 0.0010 | Train Accuracy : 100.00%
Epoch : 2387 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2388 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2389 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2390 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2391 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2392 | Loss : 0.0000 | Train Accuracy : 

Epoch : 2522 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2523 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2524 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2525 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2526 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2527 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2528 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2529 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2530 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2531 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2532 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2533 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2534 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2535 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2536 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2537 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2538 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2539 | Loss : 0.0014 | Train Accuracy : 

Epoch : 2672 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2673 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2674 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2675 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2676 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2677 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2678 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2679 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2680 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2681 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2682 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2683 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2684 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2685 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2686 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2687 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2688 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2689 | Loss : 0.0000 | Train Accuracy : 

Epoch : 2819 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2820 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2821 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2822 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2823 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2824 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2825 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2826 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2827 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2828 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2829 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2830 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2831 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2832 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2833 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2834 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2835 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2836 | Loss : 0.0000 | Train Accuracy : 

Epoch : 2969 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2970 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2971 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2972 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2973 | Loss : 0.0009 | Train Accuracy : 100.00%
Epoch : 2974 | Loss : 0.0003 | Train Accuracy : 100.00%
Epoch : 2975 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2976 | Loss : 0.0008 | Train Accuracy : 100.00%
Epoch : 2977 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2978 | Loss : 0.0017 | Train Accuracy : 100.00%
Epoch : 2979 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2980 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2981 | Loss : 0.0002 | Train Accuracy : 100.00%
Epoch : 2982 | Loss : 0.0000 | Train Accuracy : 100.00%
Epoch : 2983 | Loss : 0.0009 | Train Accuracy : 100.00%
Epoch : 2984 | Loss : 0.0004 | Train Accuracy : 100.00%
Epoch : 2985 | Loss : 0.0001 | Train Accuracy : 100.00%
Epoch : 2986 | Loss : 0.0000 | Train Accuracy : 

## Test the model

In [205]:
simpleModel.eval()
testX = np.array([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
predicts = simpleModel(torch.from_numpy(testX).float().to(device))

In [206]:
_,res = torch.max(predicts,1)
res

tensor([0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1,
        2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1,
        0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1,
        0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1,
        0, 0, 1, 2], device='cuda:0')

## Show the result

In [123]:
def fizz_buzz_decode(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

In [207]:
predictions = [fizz_buzz_decode(i,prediction) for (i,prediction) in zip(range(1,101),res)]
print(predictions)

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz', 'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', 'fizz', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


In [208]:
real_res = np.array([fizz_buzz_encode(i) for i in range(1, 101)])
real = [fizz_buzz_decode(i,prediction) for (i,prediction) in zip(range(1,101),real_res)]
print(real)

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz', 'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', 'fizz', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


In [209]:
# 计算测试集的准确率
right = 0
for i,j in zip(real,predictions):
    if i==j:
        right = right + 1
    else:
        print('real:{} / pred:{}'.format(i,j))
print('Acc : {}%'.format(right))

Acc : 100%


In [189]:
# 预测单个的值
simpleModel.eval()
simpleModel(torch.from_numpy(binary_encode(100,10)[None]).float().to(device))

tensor([[  3.7646,  -5.7268,  12.7268, -21.9920]], device='cuda:0',
       grad_fn=<AddmmBackward>)